In [ ]:
# Task 1: Data Preprocessing without External Libraries

def preprocess_insurance_data(csv_file_path):
    """
    Preprocesses insurance claims data without using external libraries.
    
    Args:
        csv_file_path (str): Path to the CSV file
        
    Returns:
        list: List of dictionaries representing cleaned data
    """
    
    # Read the CSV file
    with open(csv_file_path, 'r') as file:
        lines = file.readlines()
    
    # Parse header
    header = lines[0].strip().split(',')
    
    # Initialize cleaned data list
    cleaned_data = []
    
    # Process each row
    for i in range(1, len(lines)):
        row = lines[i].strip().split(',')
        
        # Create dictionary for current row
        row_dict = {}
        
        for j, column in enumerate(header):
            # Handle cases where row might have fewer columns
            if j < len(row):
                value = row[j].strip()
                
                # Handle missing values
                if value == '' or value.lower() == 'null' or value.lower() == 'none':
                    row_dict[column] = None
                else:
                    # Try to convert numeric columns
                    if column in ['CLAIM_AMOUNT', 'PREMIUM_COLLECTED', 'PAID_AMOUNT']:
                        try:
                            row_dict[column] = float(value)
                        except ValueError:
                            row_dict[column] = None
                    else:
                        row_dict[column] = value
            else:
                row_dict[column] = None
        
        # Remove rows with missing data (CLAIM_ID, CUSTOMER_ID)
        if row_dict.get('CLAIM_ID') and row_dict.get('CUSTOMER_ID'):
            # Handle invalid claim amounts (negative values)
            if row_dict.get('CLAIM_AMOUNT') is not None and row_dict['CLAIM_AMOUNT'] < 0:
                row_dict['CLAIM_AMOUNT'] = None
            
            # Handle invalid premium amounts (negative values)
            if row_dict.get('PREMIUM_COLLECTED') is not None and row_dict['PREMIUM_COLLECTED'] < 0:
                row_dict['PREMIUM_COLLECTED'] = None
            
            # Handle invalid paid amounts (negative values)
            if row_dict.get('PAID_AMOUNT') is not None and row_dict['PAID_AMOUNT'] < 0:
                row_dict['PAID_AMOUNT'] = None
            
            # Standardize city names
            if row_dict.get('CITY'):
                row_dict['CITY'] = row_dict['CITY'].upper().strip()
            
            cleaned_data.append(row_dict)
    
    return cleaned_data

# Test the preprocessing function
print("=== Task 1: Data Preprocessing ===")
print("Function created successfully. Testing with sample data...")

# Load and preprocess the data
cleaned_data = preprocess_insurance_data('Insurance_auto_data.csv')
print(f"Total records processed: {len(cleaned_data)}")
print(f"Sample record: {cleaned_data[0]}")

# Task 2: City Closure Analysis

def analyze_city_performance(data):
    """
    Analyzes performance metrics for each city to recommend closure.
    
    Args:
        data (list): Cleaned insurance data
        
    Returns:
        dict: Analysis results for each city
    """
    
    city_metrics = {}
    
    # Initialize metrics for each city
    cities = ['PUNE', 'KOLKATA', 'RANCHI', 'GUWAHATI']
    for city in cities:
        city_metrics[city] = {
            'total_claims': 0,
            'total_claim_amount': 0,
            'total_premium_collected': 0,
            'total_paid_amount': 0,
            'rejected_claims': 0,
            'avg_claim_amount': 0,
            'avg_premium': 0,
            'claim_ratio': 0,  # Paid amount / Claim amount
            'profitability': 0,  # Premium - Paid amount
            'rejection_rate': 0
        }
    
    # Calculate metrics for each city
    for record in data:
        city = record.get('CITY')
        if city and city in cities:
            city_metrics[city]['total_claims'] += 1
            
            # Add claim amount
            if record.get('CLAIM_AMOUNT') is not None:
                city_metrics[city]['total_claim_amount'] += record['CLAIM_AMOUNT']
            
            # Add premium collected
            if record.get('PREMIUM_COLLECTED') is not None:
                city_metrics[city]['total_premium_collected'] += record['PREMIUM_COLLECTED']
            
            # Add paid amount
            if record.get('PAID_AMOUNT') is not None:
                city_metrics[city]['total_paid_amount'] += record['PAID_AMOUNT']
            
            # Check for rejections
            if record.get('PAID_AMOUNT') == 0 and record.get('REJECTION_REMARKS'):
                city_metrics[city]['rejected_claims'] += 1
    
    # Calculate derived metrics
    for city in cities:
        metrics = city_metrics[city]
        
        if metrics['total_claims'] > 0:
            # Average claim amount
            if metrics['total_claim_amount'] > 0:
                metrics['avg_claim_amount'] = metrics['total_claim_amount'] / metrics['total_claims']
            
            # Average premium
            if metrics['total_premium_collected'] > 0:
                metrics['avg_premium'] = metrics['total_premium_collected'] / metrics['total_claims']
            
            # Claim ratio (efficiency)
            if metrics['total_claim_amount'] > 0:
                metrics['claim_ratio'] = metrics['total_paid_amount'] / metrics['total_claim_amount']
            
            # Profitability
            metrics['profitability'] = metrics['total_premium_collected'] - metrics['total_paid_amount']
            
            # Rejection rate
            metrics['rejection_rate'] = metrics['rejected_claims'] / metrics['total_claims']
    
    return city_metrics

# Perform city analysis
print("\n=== Task 2: City Performance Analysis ===")
city_analysis = analyze_city_performance(cleaned_data)

# Display results
for city, metrics in city_analysis.items():
    print(f"\n{city}:")
    print(f"  Total Claims: {metrics['total_claims']}")
    print(f"  Total Premium Collected: ₹{metrics['total_premium_collected']:,.2f}")
    print(f"  Total Paid Amount: ₹{metrics['total_paid_amount']:,.2f}")
    print(f"  Profitability: ₹{metrics['profitability']:,.2f}")
    print(f"  Rejection Rate: {metrics['rejection_rate']:.2%}")
    print(f"  Claim Ratio: {metrics['claim_ratio']:.2%}")

# Recommendation logic
def recommend_city_closure(city_metrics):
    """
    Recommends which city to close based on multiple factors.
    """
    scores = {}
    
    for city, metrics in city_metrics.items():
        # Score based on multiple factors (lower is worse)
        profitability_score = metrics['profitability']
        rejection_penalty = metrics['rejection_rate'] * 1000  # Penalty for high rejections
        claim_efficiency = metrics['claim_ratio'] * 100  # Reward for efficient claim processing
        
        # Overall score (higher is better)
        scores[city] = profitability_score + claim_efficiency - rejection_penalty
    
    # Find city with lowest score
    worst_city = min(scores.keys(), key=lambda x: scores[x])
    
    return worst_city, scores

recommended_closure, city_scores = recommend_city_closure(city_analysis)
print(f"\n=== RECOMMENDATION ===")
print(f"Recommended city for closure: {recommended_closure}")
print("\nCity Scores (higher is better):")
for city, score in sorted(city_scores.items(), key=lambda x: x[1], reverse=True):
    print(f"  {city}: {score:.2f}")

## Task 3: Debug and Fix Rejection Classifier

# ANALYSIS OF ORIGINAL BUGGY CODE (commented out due to syntax errors)
print("=== ANALYSIS OF ORIGINAL BUGGY CODE ===")
print("The original code contained the following bugs:")

# Dictionary for mapping (this part was correct)
REJECTION_REASONS_MAP = {
    "Fake_document": "Fake_document",
    "Not_Covered": "Not_Covered",
    "Policy_expired": "Policy_expired"
}

# BUGGY CODE ANALYSIS


# FIXED VERSIONS OF ALL FUNCTIONS
print("\n=== FIXED CODE (errors corrected) ===")

# Function 1 - FIXED
def handle_error(error_message):
    print(f"Error: {error_message}")  # FIXED: correct variable name
    return "Error"  # FIXED: correct spelling

# Function 2 - FIXED
def contains_rejection_reason(rejection_text, reason):  # FIXED: parameter name
    try:
        if rejection_text and isinstance(rejection_text, str):  # FIXED: consistent variable name
            return reason.lower() in rejection_text.lower()  # FIXED: case-insensitive search
    except Exception as e:  # FIXED: correct exception name
        handle_error(f"Error in contains_rejection_reason: {str(e)}")
        return False
    return False

# Function 3 - FIXED
def map_rejection_reason(rejection_text):
    try:
        if rejection_text and isinstance(rejection_text, str):
            for reason, rejection_class in REJECTION_REASONS_MAP.items():  # FIXED: "in" instead of "is"
                if contains_rejection_reason(rejection_text, reason):  # FIXED: function call
                    return rejection_class
            return "Unknown"  
        else:
            return "NoRemark"
    except Exception as e:
        handle_error(f"Error in map_rejection_reason: {str(e)}")
        return "Error"  # FIXED: spelling

# Function 4 - MAIN FIXED FUNCTION
def complex_rejection_classifier(remark_text):
    try:
        # FIXED: Check if it's NOT a string OR if it's empty
        if not isinstance(remark_text, str) or len(remark_text.strip()) == 0:
            return "Invalid Remark"

        # Check for each rejection reason
        fake_doc = contains_rejection_reason(remark_text, "Fake_document")  # FIXED: function name
        not_covered = contains_rejection_reason(remark_text, "Not_Covered")
        policy_expired = contains_rejection_reason(remark_text, "Policy_expired")

        if fake_doc:
            return "Fake_document"  # FIXED: complete "return" keyword
        elif not_covered:
            return "Not_Covered"
        elif policy_expired:
            return "Policy_expired"
        else:
            # Unknown or null remarks
            return map_rejection_reason(remark_text)
    except Exception as e:
        handle_error(f"Error in complex_rejection_classifier: {str(e)}")
        return "Error"  # FIXED: spelling

# SUMMARY OF BUGS FOUND AND FIXED:
print("\n=== BUGS IDENTIFIED AND FIXED ===")
bugs_fixed = [
    "1. handle_error(): 'error_messages' → 'error_message' (variable name mismatch)",
    "2. handle_error(): 'Errror' → 'Error' (typo)",
    "3. contains_rejection_reason(): 'rejection_texts' → 'rejection_text' (parameter name)",
    "4. contains_rejection_reason(): 'Exceptions' → 'Exception' (wrong exception type)",
    "5. contains_rejection_reason(): Added case-insensitive search",
    "6. map_rejection_reason(): 'is' → 'in' (wrong operator in for loop)",
    "7. map_rejection_reason(): 'Errror' → 'Error' (typo)",
    "8. complex_rejection_classifier(): 'isinstance(remark_text, int)' → 'isinstance(remark_text, str)' (wrong type check)",
    "9. complex_rejection_classifier(): 'contains_rejection_reasons' → 'contains_rejection_reason' (function name typo)",
    "10. complex_rejection_classifier(): 'retur' → 'return' (incomplete keyword)",
    "11. complex_rejection_classifier(): 'Errror' → 'Error' (typo)"
]

for bug in bugs_fixed:
    print(f"   {bug}")

# Test the fixed function with sample data
print("\n=== TESTING FIXED FUNCTION ===")
test_cases = [
    "Rejection reason: Policy_expired in verification.",
    "Fake_document reason led to rejection.",
    "Submission error - Not_Covered found.",
    "Policy rejected: Policy_expired noted.",
    "System flagged this claim: Policy_expired.",
    "Policy rejected: Fake_document noted.",
    "",
    None,
    123  # Non-string input
]

print("Testing with sample rejection remarks:")
for i, test_case in enumerate(test_cases):
    try:
        result = complex_rejection_classifier(test_case)
        print(f"{i+1}. Input: {repr(test_case)} → Output: '{result}'")
    except Exception as e:
        print(f"{i+1}. Input: {repr(test_case)} → Error: {str(e)}")

# Apply the fixed function to create REJECTION_CLASS column
print("\n=== Task 3: Rejection Classification ===")

# Add rejection classification to each record
for record in cleaned_data:
    remark = record.get('REJECTION_REMARKS')
    if remark and remark.strip():
        record['REJECTION_CLASS'] = complex_rejection_classifier(remark)
    else:
        record['REJECTION_CLASS'] = 'No Remark'

# Analyze rejection classifications
rejection_stats = {}
for record in cleaned_data:
    rejection_class = record['REJECTION_CLASS']
    if rejection_class not in rejection_stats:
        rejection_stats[rejection_class] = 0
    rejection_stats[rejection_class] += 1

print("Rejection Classification Summary:")
for classification, count in sorted(rejection_stats.items()):
    print(f"  {classification}: {count}")

# Show sample classifications
print("\nSample Classifications:")
rejection_samples = []
for record in cleaned_data:
    if record.get('REJECTION_REMARKS') and record['REJECTION_REMARKS'].strip():
        rejection_samples.append({
            'Original': record['REJECTION_REMARKS'],
            'Classification': record['REJECTION_CLASS']
        })

for i, sample in enumerate(rejection_samples[:5]):
    print(f"{i+1}. '{sample['Original']}' → {sample['Classification']}")

## Summary and Insights

print("\n=== FINAL SUMMARY ===")
print(f"1. Data Processing: Successfully processed {len(cleaned_data)} insurance claims")
print(f"2. City Analysis: Recommended closure of {recommended_closure} based on profitability and efficiency metrics")
print(f"3. Rejection Classification: Categorized rejection remarks into {len(rejection_stats)} categories")

# Additional insights
total_claims = len(cleaned_data)
total_premium = sum(record.get('PREMIUM_COLLECTED', 0) or 0 for record in cleaned_data)
total_paid = sum(record.get('PAID_AMOUNT', 0) or 0 for record in cleaned_data)
overall_profitability = total_premium - total_paid

print(f"\nOverall Business Metrics:")
print(f"- Total Claims: {total_claims}")
print(f"- Total Premium Collected: ₹{total_premium:,.2f}")
print(f"- Total Amount Paid: ₹{total_paid:,.2f}")
print(f"- Overall Profitability: ₹{overall_profitability:,.2f}")
print(f"- Average Profit per Claim: ₹{overall_profitability/total_claims:,.2f}")

=== Task 1: Data Preprocessing ===
Function created successfully. Testing with sample data...
Total records processed: 100
Sample record: {'CLAIM_ID': 'CLM100021', 'CLAIM_DATE': '2025-04-01', 'CUSTOMER_ID': 'CUST14285', 'CLAIM_AMOUNT': 10419.0, 'PREMIUM_COLLECTED': 2198.59, 'PAID_AMOUNT': 6964.46, 'CITY': 'PUNE', 'REJECTION_REMARKS': None}

=== Task 2: City Performance Analysis ===

PUNE:
  Total Claims: 37
  Total Premium Collected: ₹369,254.79
  Total Paid Amount: ₹1,093,273.28
  Profitability: ₹-724,018.49
  Rejection Rate: 8.11%
  Claim Ratio: 70.81%

KOLKATA:
  Total Claims: 16
  Total Premium Collected: ₹140,279.78
  Total Paid Amount: ₹636,392.57
  Profitability: ₹-496,112.79
  Rejection Rate: 0.00%
  Claim Ratio: 75.06%

RANCHI:
  Total Claims: 17
  Total Premium Collected: ₹148,858.60
  Total Paid Amount: ₹401,142.67
  Profitability: ₹-252,284.07
  Rejection Rate: 11.76%
  Claim Ratio: 70.34%

GUWAHATI:
  Total Claims: 24
  Total Premium Collected: ₹261,314.84
  Total Paid Amo